# Intake-STAC + NASA CMR

NASA's Common Metadata Repository (CMR) now has a STAC endpoint https://github.com/nasa/cmr-stac. In theory we can use Intake-STAC to browse NASA's entire data catalog! 

**Warning: This example notebook is very experimental and mostly for testing purposes.**

In [ ]:
%matplotlib inline
import intake

In [ ]:
# STAC catalogs sometimes don't have 'mimetype per asset'
# This blocks silences warnings related to this case
# https://github.com/intake/intake-stac/issues/59
import warnings
warnings.simplefilter('ignore')

In [ ]:
# for development
%load_ext autoreload
%autoreload 2

In [ ]:
import satsearch
satsearch.__version__

In [ ]:
# requires sat-search>0.3.0
!sat-search search --url=https://cmr.earthdata.nasa.gov/cmr-stac/ASF

In [ ]:
cat = intake.open_stac_catalog('https://cmr.earthdata.nasa.gov/cmr-stac/ASF/collections')

In [ ]:
[x['short_name'] for x in cat.metadata['collections']]

In [ ]:
import pandas as pd
band_info = pd.DataFrame(cat.metadata['collections'])
band_info

In [ ]:
# Easily get all endpoints and collections?
#import requests
URL = 'https://cmr.earthdata.nasa.gov/cmr-stac/ASF'
results = satsearch.Search.search(url=URL,
                                  collections=['C1595422627-ASF']
                                  )
items = results.items()
print('%s items' % len(items))

In [ ]:
catalog = intake.open_stac_item_collection(items)

In [ ]:
# NOTE: import statement required for dynamic=True
import hvplot.xarray
intake.gui.add(catalog)
intake.gui

In [ ]:
# Item ids not so informative!
#list(catalog)
item = catalog['G1636018550-ASF']

In [ ]:
list(item)

In [ ]:
item['browse'].metadata

In [ ]:
item['data']

In [ ]:
# Need to figure out authentication
# https://github.com/intake/intake-stac/issues/60
da = item['data'].to_dask()

In [ ]:
# NOTE: this requires NASA URS Earthdata credentials in a .netrc file
#!wget https://grfn.asf.alaska.edu/door/download/S1-GUNW-A-R-087-tops-20141023_20141011-153856-27545N_25464N-PP-1a1a-v2_0_2.nc

In [ ]:
!ls -ltrh S1-GUNW-A-R-087-tops-20141023_20141011-153856-27545N_25464N-PP-1a1a-v2_0_2.nc

In [ ]:
!gdalinfo S1-GUNW-A-R-087-tops-20141023_20141011-153856-27545N_25464N-PP-1a1a-v2_0_2.nc

In [ ]:
import xarray as xr
localFile = 'S1-GUNW-A-R-087-tops-20141023_20141011-153856-27545N_25464N-PP-1a1a-v2_0_2.nc'
da = xr.open_dataset(localFile,
                     group='/science/grids/data')
da